In [ ]:
import Securities as sec
import gspread as gs
import warnings
import sys
warnings.filterwarnings('ignore')

In [ ]:
tech_tickers = ["AAPL","AMZN","TSLA", "NVDA", "MSFT", "GOOG","INTC", "DELL"]

In [ ]:
securities = sec.Returns_analysis(tech_tickers, start_date = "2005-1-1")

In [ ]:
securities.download_stocks(column_name= "Adj Close", fill_method = "forward")

In [ ]:
securities.normalization()

In [ ]:
securities.calculate_returns("Simple")

In [ ]:
securities.individual_return()

In [ ]:
securities.individual_risk_factor()